In [1]:
import pandas as pd
import re
import json

# Load configuration
# with open('config.json', 'r') as config_file:
#     config = json.load(config_file)


excel_file = "Copy of SNF-B-Rates 2022 - 2023 - Revised.xlsx"
sheet_name_1 = "Special Rates LTCs"


# sheet_name_2  = "2022 FS NF-B Rates"
# hrl_file_path = '/Users/kirankumarvelinedi/Documents/HRP/LTC Components/FeeComponent.LTCSNF-B.1800-01-01.a.hrl'
# output_file_path = "Updated_2022_FeeComponent.LTCSNF-B.1800-01-01.a.hrl"

sheet_name_2  = "2023 FS NF-B UPDATED"
hrl_file_path = '/Users/kirankumarvelinedi/Documents/HRP/LTC Components/FeeComponent.LTCSNF-B.1800-01-01.a.hrl'
output_file_path = "Updated_2023_FeeComponent.LTCSNF-B.1800-01-01.a.hrl"


# sheet_name_2  = "2022 FS NF-B Rates"
# hrl_file_path = '/Users/kirankumarvelinedi/Documents/HRP/LTC Components/FeeComponent.LTCRatesNSFB.2022-08-01.a.hrl'
# output_file_path = "Updated_2022_FeeComponent.LTCRatesNSFB.2022-08-01.a.hrl"

# sheet_name_2  = "2023 FS NF-B UPDATED"
# hrl_file_path = '/Users/kirankumarvelinedi/Documents/HRP/LTC Components/FeeComponent.LTCRatesNSFB.2023-05-12.a.hrl'
# output_file_path = "Updated_2023_FeeComponent.LTCRatesNSFB.2023-05-12.a.hrl"

# sheet_name_2  = "2023 FS NF-B UPDATED"
# hrl_file_path = '/Users/kirankumarvelinedi/Documents/HRP/LTC Components/FeeComponent.LTCRatesNSFB.2023-08-01.a.hrl'
# output_file_path = "Updated_2023_FeeComponent.LTCRatesNSFB.2023-08-01.a.hrl"

# sheet_name_2  = "2022 FS NF-B Rates"
# hrl_file_path = '/Users/kirankumarvelinedi/Documents/HRP/LTC Components/FeeComponent.LTCRatesNSFB.1800-01-01.a.hrl'
# output_file_path = "Updated_2023_FeeComponent.LTCRatesNSFB.1800-01-01.a.hrl"




# sheet_name_2  = "2023 FS NF-B UPDATED"
# hrl_file_path = '/Users/kirankumarvelinedi/Documents/HRP/LTC Components/FeeComponent.LTCSNF-B.1800-01-01.a.hrl'
# output_file_path = "Updated_2023_FeeComponent.LTCSNF-B.1800-01-01.a.hrl"




columns = [
      "HCAI ID",
      "NPI Provider ID",
      "Facility Name",
      "Rate Published/Updated Date",
      "Former NPI",
      "Accommodation Code 01 (FS/NF-B Regular Services)",
      "Accommodation Codes 02 and 03 (FS/NF-B Leave Days)",
      "Accommodation Code 11 (STP Regular Services)",
      "Accommodation Code 12 (STP Leave Days)"
    ]
columns_to_extract = ["Accommodation Code 01 (FS/NF-B Regular Services)",
      "Accommodation Codes 02 and 03 (FS/NF-B Leave Days)",
      "Accommodation Code 11 (STP Regular Services)",
      "Accommodation Code 12 (STP Leave Days)"]

In [2]:

# Load the Excel file and read the first sheet for Special Rates LTCs
df_vendor = pd.read_excel(excel_file, sheet_name=sheet_name_1)

# Create a dictionary mapping NPI to Vendor ID
npi_to_vendor = dict(zip(df_vendor['NPI'], df_vendor['Vendor ID']))

# Load the Excel file and read the second sheet, skipping the first row
df_accommodation = pd.read_excel(excel_file, sheet_name=sheet_name_2, skiprows=2)

if "2023" in sheet_name_2:
      df_accommodation.drop(df_accommodation.columns[4], axis=1, inplace=True)
        
# Select only the necessary columns
df_accommodation = df_accommodation.iloc[:, :9]

df_accommodation.columns = columns

# Drop duplicate NPI Provider ID values
df_accommodation = df_accommodation.drop_duplicates(subset=['NPI Provider ID'])

# Create a dictionary mapping NPI to relevant columns
npi_to_accommodation = df_accommodation.set_index('NPI Provider ID')[columns_to_extract].to_dict('index')

# Read the text file
with open(hrl_file_path, 'r') as file:
    text_lines = file.readlines()
    

hrl_npi_lst = []
hrl_npl_special_lst = []
hrl_npl_accommod_lst = []

# Function to process each line
def process_line(line):
    # Find NPI value in the line
    npi_value = re.findall(r'\b\d{10}\b', line)
    if len(npi_value) > 0:
        npi_num = int(npi_value[0])
        hrl_npi_lst.append(npi_num)
        if npi_num in npi_to_vendor:
            hrl_npl_special_lst.append(npi_num)
            vendor_id = npi_to_vendor[npi_num]
            # Insert the vendor information 
#             line = re.sub(r'(note)', f'and supplier {vendor_id} \\1', line)
            line = re.sub(r'\) the allowed fee', rf') and supplier "{vendor_id}" the allowed fee', line)
        
        if npi_num in npi_to_accommodation:
            hrl_npl_accommod_lst.append(npi_num)
            accommodation_info = npi_to_accommodation[npi_num]
            # Check for specific revenue codes in the line
            if re.search(r'\b0001\b', line):
                accommodation_details = accommodation_info["Accommodation Code 01 (FS/NF-B Regular Services)"]
            elif re.search(r'\b0002\b|\b0003\b', line):
                accommodation_details = accommodation_info["Accommodation Codes 02 and 03 (FS/NF-B Leave Days)"]
            elif re.search(r'\b0011\b', line):
                accommodation_details = accommodation_info["Accommodation Code 11 (STP Regular Services)"]
            elif re.search(r'\b0012\b', line):
                accommodation_details = accommodation_info["Accommodation Code 12 (STP Leave Days)"]
            else:
                accommodation_details = None

            if accommodation_details is not None:
                # Insert the accommodation details after 'the allowed fee is'
                line = re.sub(r'(the allowed fee is \$)(\d+\.\d+)', lambda m: f'{m.group(1)}{float(accommodation_details):.2f}', line)
        
        return line
    else:
        return line

# Process each line of the text file
updated_lines = [process_line(line) for line in text_lines]

# Write the updated lines back to a new text file
with open(output_file_path, 'w') as file:
    file.writelines(updated_lines)

print("File updated successfully!\n")

File updated successfully!



In [3]:
npi_vendor_count = df_vendor['NPI'].count()
npi_vendor_distinct = df_vendor['NPI'].nunique()
duplicates = list(df_vendor[df_vendor.duplicated(subset='NPI', keep=False)]['NPI'].unique())

print(f"Total     NPI Records in Special Rates LTCs sheet : {npi_vendor_count}")
print(f"Distinct  NPI Records in Special Rates LTCs sheet : {npi_vendor_distinct}")
print(f"Duplicate NPI Records in Special Rates LTCs sheet : {duplicates}\n")

Total     NPI Records in Special Rates LTCs sheet : 261
Distinct  NPI Records in Special Rates LTCs sheet : 260
Duplicate NPI Records in Special Rates LTCs sheet : [1821118878]



In [4]:
print(f"'{sheet_name_2}' vs '{hrl_file_path}'\n")

print(f" Total NPI's Distinct Count in HRL file                   : {len(list(set(hrl_npi_lst)))}\n")

print(f" HRL-NPI's matched   count  to {sheet_name_1} sheet   : {len(list(set(hrl_npl_special_lst)))}")
print(f" HRL-NPI's unmatched values to {sheet_name_1} sheet   : {list((set(hrl_npi_lst) - set(hrl_npl_special_lst)))}\n")

print(f" HRL-NPI's matched   count  to {sheet_name_2} sheet : {len(list(set(hrl_npl_accommod_lst)))}")
print(f" HRL-NPI's unmatched values to {sheet_name_2} sheet : {list((set(hrl_npi_lst) - set(hrl_npl_accommod_lst)))}")

'2023 FS NF-B UPDATED' vs '/Users/kirankumarvelinedi/Documents/HRP/LTC Components/FeeComponent.LTCSNF-B.1800-01-01.a.hrl'

 Total NPI's Distinct Count in HRL file                   : 249

 HRL-NPI's matched   count  to Special Rates LTCs sheet   : 248
 HRL-NPI's unmatched values to Special Rates LTCs sheet   : [1114119054]

 HRL-NPI's matched   count  to 2023 FS NF-B UPDATED sheet : 247
 HRL-NPI's unmatched values to 2023 FS NF-B UPDATED sheet : [1689653289, 1821118878]


###                                                  OUTLINERS IN 2022 SHEET  

In [9]:


sheet_name_1  = "Special Rates LTCs"
df_npi_vendor = pd.read_excel(excel_file, sheet_name=sheet_name_1)


# sheet_name_2 = "2022 FS NF-B Rates"

sheet_name_2 =  "2023 FS NF-B UPDATED"
df_accommodation = pd.read_excel(excel_file, sheet_name=sheet_name_2, skiprows=1)

if "2023" in sheet_name_2:
      df_accommodation.drop(df_accommodation.columns[4], axis=1, inplace=True)
        
# display(df_accommodation)
df_accommodation = df_accommodation.iloc[:, :9]        
        
npi_lst = df_npi_vendor['NPI'].to_list()


columns=["OSHPD ID",
"National Provider Identifier (NPI)",
"Owner", 
"Loc", 
"Facility Name",
"Accommodation Code 01",
"Accommodation Codes 02 and 03",
"Accommodation Code 11",
"Accommodation Code 12"]
df_accommodation.columns = columns



filtered_df = df_accommodation[df_accommodation['National Provider Identifier (NPI)'].isin(npi_lst)]
matched = filtered_df['National Provider Identifier (NPI)'].to_list()
unmatched = list(set(npi_lst) - set(matched))
print(f"The Outlier in '{sheet_name_2}'\n")
print(unmatched)
filtered_df

The Outlier in '2023 FS NF-B UPDATED'

[1689653289, 1699747659, 1346295755, 1255656781, 1871661967, 1518227057, 1942335062, 1700806585, 1518913177, 1881696474, 1184670747, 1821118878]


,OSHPD ID,National Provider Identifier (NPI),Owner,Loc,Facility Name,Accommodation Code 01,Accommodation Codes 02 and 03,Accommodation Code 11,Accommodation Code 12
64,206071044,1831235290,WINDSOR ROSEWOOD CARE CENTER,2023-09-11 00:00:00,NaN,330.03,320.79,0,0
67,206073349,1558532465,MUIR SENIOR CARE,2023-09-11 00:00:00,NaN,309.26,300.02,0,0
93,206120955,1003113705,GRANADA REHABILITATION & WELLNESS CENTER,2023-09-11 00:00:00,NaN,312.163336,302.923336,0,0
102,206154002,1114150448,THE REHABILITATION CENTER OF BAKERSFIELD,2023-09-11 00:00:00,NaN,257.08,247.84,0,0
105,206160726,1629527429,HANDFORD POST ACUTE,2023-09-11 00:00:00,NaN,247.55,238.31,0,0
...,...,...,...,...,...,...,...,...,...
1002,206304007,1891741476,LAKE FOREST NURSING CENTER,2024-04-19 00:00:00,NaN,346.01,336.77,0,0
1015,206371716,1386681286,LIFE CARE CENTER OF ESCONDIDO,2024-04-19 00:00:00,NaN,290.342095,281.102095,0,0
1016,206420499,1841351780,MISSION TERRACE CONVALESCENT HOSPITAL,2024-04-19 00:00:00,NaN,352.07,342.83,0,0
1018,206480995,1235152679,LA MARIPOSA CARE AND REHAB CENTER,2024-04-19 00:00:00,NaN,332.98,323.74,0,0
